In [114]:
import os 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import mean_squared_error
from math import sqrt


In [ ]:
%cd /content/drive/MyDrive/SINA

/content/drive/MyDrive/SINA


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# !unzip /content/drive/MyDrive/SINA/ml-latest-small.zip

Archive:  /content/drive/MyDrive/SINA/ml-latest-small.zip
   creating: ml-latest-small/
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  


In [104]:
links = pd.read_csv('/content/drive/MyDrive/SINA/ml-latest-small/links.csv')
movies = pd.read_csv('/content/drive/MyDrive/SINA/ml-latest-small/movies.csv')
ratings = pd.read_csv('/content/drive/MyDrive/SINA/ml-latest-small/ratings.csv')
tags =pd.read_csv('/content/drive/MyDrive/SINA/ml-latest-small/tags.csv')

In [105]:
# Merge movies and ratings dataframes
data = pd.merge(ratings, movies, on='movieId')

# Create a user-item matrix
matrix = data.pivot(index='userId', columns='movieId', values='rating')
mask = ~np.isnan(matrix)
matrix = matrix.fillna(0)
matrix_sparse = csr_matrix(matrix.values)


In [106]:
# Perform matrix factorization
model = TruncatedSVD(n_components=100, random_state=42)
matrix_reduced = model.fit_transform(matrix_sparse)


In [107]:
movie_id_title_map = dict(zip(movies['movieId'], movies['title']))
movie_index_id_map = dict( zip( range(len(matrix.columns)),  matrix.columns.tolist() )  )
movie_id_index_map = dict( zip( matrix.columns.tolist(), range(len(matrix.columns)) )  )

In [110]:
def recommend_movies(user_id, top_n=3):
    # Get the user's predicted ratings
    # user_ratings = matrix_reduced[user_id - 1]
    user_ratings = model.inverse_transform(matrix_reduced)[user_id-1,:]

    # Get the indices of the top-rated movies
    top_movie_indices = np.argsort(-user_ratings)[:top_n]

    # Get the movie IDs, titles, and predicted ratings of the top-rated movies
    top_movie_ids = []
    top_movie_titles = []
    top_movie_ratings = []
    for index in top_movie_indices:
        movie_id = movie_index_id_map[index]
        if movie_id in movie_id_title_map:
            top_movie_ids.append(movie_id)
            top_movie_titles.append(movie_id_title_map[movie_id])
            rating = min(user_ratings[index], 5.0)  # Cap the rating at 5.0 if it exceeds
            top_movie_ratings.append(rating)

    return pd.DataFrame({'userID': [user_id] * len(top_movie_ids), 'title': top_movie_titles, 'movieId': top_movie_ids, 'predicted_rating': top_movie_ratings})


In [111]:
user_id = 1  # Example user ID
top_movies = recommend_movies(user_id)


In [112]:
top_movies

,userID,title,movieId,predicted_rating
0,1,Jurassic Park (1993),480,5.0
1,1,"Usual Suspects, The (1995)",50,5.0
2,1,Reservoir Dogs (1992),1089,5.0


In [115]:
# Make predictions on the test set and calculate RMSE

predictions = model.inverse_transform(matrix_reduced)
rmse = sqrt(mean_squared_error(predictions[mask], matrix.values[mask]))

print("RMSE:", rmse)

RMSE: 1.5816437793295866
